In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from monty.serialization import loadfn, dumpfn

In [3]:
from ase.filters import FrechetCellFilter
from ase.optimize import LBFGS
from pymatgen.io.ase import AseAtomsAdaptor
from orb_models.forcefield import pretrained
from orb_models.forcefield.calculator import ORBCalculator
from fairchem.core import pretrained_mlip, FAIRChemCalculator
# from eqnorm.calculator import EqnormCalculator
# from mace.calculators import mace_mp
# from mattersim.forcefield import MatterSimCalculator
# from sevenn.calculator import SevenNetCalculator


/home/atshaam/miniconda3/envs/proj_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL = "uma" # "orb" or "uma" or "mace" or "seven" or "mattersim" or "eqnorm"

In [5]:
supercell_size = 10

In [6]:
# orb_ff = pretrained.orb_v3_conservative_inf_omat(device="cuda",precision="float32-high")  # or "float32-highest" / "float64
# orb_calc = ORBCalculator(orb_ff, device="cuda")

uma_ff = pretrained_mlip.get_predict_unit("uma-m-1p1", device="cuda")
uma_calc = FAIRChemCalculator(uma_ff, task_name="omat")

# eqnorm_calc = EqnormCalculator(model_name='eqnorm', model_variant='eqnorm-mptrj', device='cuda')


# mace_calc = mace_mp(model="MACE-matpes-r2scan-omat-ft.model", dispersion=False, device='cuda')
# seven_calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')

# mattersim_calc = MatterSimCalculator(device="cuda")

MODELS = {
    # "orb":orb_calc,
    "uma": uma_calc,
    # # "seven":seven_calc,
    # "mattersim":mattersim_calc,
    # "eqnorm":eqnorm_calc
    # "mace": mace_calc
}

/home/atshaam/miniconda3/envs/proj_env/lib/python3.11/site-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


In [7]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [8]:
def relax_structure(structure): # takes in pymatgen structure
    atoms = structure.to_ase_atoms()
    atoms.calc = MODELS[MODEL]
    
    opt = LBFGS(FrechetCellFilter(atoms))# Delete this FrechetCellFilter when doing the defect structures, need FrechetCellFilter when doing competing phases
    opt.run(0.01, 1000)
    
    structure1 = AseAtomsAdaptor.get_structure(atoms)
    energy = atoms.get_potential_energy()

    return structure1, energy 

In [9]:
from doped.chemical_potentials import CompetingPhases
from pymatgen.entries.computed_entries import ComputedEntry
from doped.chemical_potentials import CompetingPhasesAnalyzer
from tqdm import tqdm
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility

In [10]:
from pymatgen.core import Element

MP_U_VALUES = {
    "Co": 3.32,
    "Cr": 3.7,
    "Fe": 5.3,
    "Mn": 3.98,
    "Mo": 4.38,
    "Ni": 6.2,
    "V": 3.25,
    "W": 6.2,
}

from pymatgen.core import Species

def add_parameters(entry):
    parameters = {}
    if any([Element(el) in entry.composition.elements for el in MP_U_VALUES]):
        parameters["run_type"] = "GGA+U"
        parameters["hubbards"] = {el.name: MP_U_VALUES.get(el.symbol, 0.0) for el in entry.composition.elements}
        parameters["is_hubbard"] = True
    else:
        parameters["run_type"] = "GGA"
        parameters["hubbards"] = None
        parameters["is_hubbard"] = False
    entry.parameters = parameters
    return entry

In [11]:
calculated_entries = {}

def calc_competing_phase(comp): # Pass in formula of species
    cp = CompetingPhases(composition=comp, extrinsic='Cr', energy_above_hull=0.05)
    
    entries = []
    for entry in tqdm(cp.entries):
        key =  entry.data['material_id'] + entry.structure.composition.reduced_formula
        
        if key in calculated_entries:
            new_entry = calculated_entries[key]
        else:
            try:
                relaxed_structure, energy = relax_structure(entry.structure)
            except ValueError as e:
                print(f"Skipping structure due to error: {e}")
                continue
            new_entry = ComputedEntry(
                relaxed_structure.composition,
                energy, 
                data = {'material_id':entry.data['material_id']}
            )
            new_entry.structure = relaxed_structure   # IMPORTANT!
            new_entry = add_parameters(new_entry)     # IMPORTANT!
            calculated_entries[key] = new_entry 
        entries.append(new_entry)
    compat = MaterialsProject2020Compatibility(check_potcar=False, check_potcar_hash=False)
    new_entries = compat.process_entries(entries, inplace=False, on_error="warn")
    return cp, new_entries 

In [12]:
def calc_competing_phase_analysis(comp, entries):
    
    cpa = CompetingPhasesAnalyzer(composition=comp, entries=entries)
    
    return cpa.chempots, cpa.unstable_host

In [13]:
relaxed_defect = loadfn(f'../../data/cr_above_hull/{MODEL}_corrected_doped_defect_energies_{supercell_size}A.json')

In [14]:
for key in relaxed_defect.keys():
    formula = relaxed_defect[key]['formula']
    cp, entries = calc_competing_phase(formula) # do you have to return cp?
    chem_pot, unstable = calc_competing_phase_analysis(formula, entries)
    if 'competing phases' not in relaxed_defect[key]:
        relaxed_defect[key]['competing phases'] = {}
    relaxed_defect[key]['competing phases']['chemical potentials'] = chem_pot
    relaxed_defect[key]['competing phases']['entries'] = entries
    relaxed_defect[key]['competing phases']['unstable'] = unstable

  0%|                                                                                            | 0/29 [00:00<?, ?it/s]W0804 19:20:55.721000 19566 site-packages/torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
W0804 19:20:56.226000 19566 site-packages/torch/_dynamo/convert_frame.py:1233] WON'T CONVERT forward /home/atshaam/miniconda3/envs/proj_env/lib/python3.11/site-packages/orb_models/forcefield/gns.py line 421 
W0804 19:20:56.226000 19566 site-packages/torch/_dynamo/convert_frame.py:1233] due to: 
W0804 19:20:56.226000 19566 site-packages/torch/_dynamo/convert_frame.py:1233] Traceback (most recent call last):
W0804 19:20:56.226000 19566 site-packages/torch/_dynamo/convert_frame.py:1233]   File "/home/atshaam/miniconda3/envs/proj_env/lib/python3.11/site-packages/torch/_dynamo/convert_frame.py", line 1164, in __call__
W0804 19:20:56.226000 19566 site-packages/torch/_dynamo/convert_frame.py:1233]     result = self._inner_convert(
W0804 19:20:56.226000 

       Step     Time          Energy          fmax
LBFGS:    0 19:21:03      -25.683929        0.044265
LBFGS:    1 19:21:03      -25.683975        0.024541
LBFGS:    2 19:21:04      -25.684004        0.008580


  3%|██▉                                                                                 | 1/29 [00:20<09:36, 20.59s/it]

       Step     Time          Energy          fmax
LBFGS:    0 19:21:04      -19.026630        0.306180
LBFGS:    1 19:21:05      -19.030092        0.225895
LBFGS:    2 19:21:05      -19.034046        0.012363
LBFGS:    3 19:21:05      -19.034058        0.000990


  7%|█████▊                                                                              | 2/29 [00:21<04:07,  9.16s/it]

...

logm result may be inaccurate, approximate err = 5.412597903985433e-13
logm result may be inaccurate, approximate err = 5.418715360578212e-13
logm result may be inaccurate, approximate err = 5.445394410354084e-13
logm result may be inaccurate, approximate err = 5.505161356027562e-13


LBFGS:   43 19:56:07     -370.801147        0.014768
LBFGS:   44 19:56:07     -370.801208        0.014989
LBFGS:   45 19:56:07     -370.801270        0.011955
LBFGS:   46 19:56:07     -370.801331        0.010928


logm result may be inaccurate, approximate err = 5.486675943678187e-13
logm result may be inaccurate, approximate err = 5.513527995269044e-13
logm result may be inaccurate, approximate err = 5.472724046026393e-13
logm result may be inaccurate, approximate err = 5.497568891117125e-13


LBFGS:   47 19:56:07     -370.801361        0.010139
LBFGS:   48 19:56:08     -370.801392        0.006729


logm result may be inaccurate, approximate err = 5.506333757542342e-13
100%|███████████████████████████████████████████████████████████████████████████████████| 43/43 [01:06<00:00,  1.55s/it]
Failed to guess oxidation states for Entry None (Al15Fe9O32). Assigning anion correction to only the most electronegative atom.
Failed to guess oxidation states for Entry None (Fe17O18). Assigning anion correction to only the most electronegative atom.
Failed to guess oxidation states for Entry None (Fe10O11). Assigning anion correction to only the most electronegative atom.
Failed to guess oxidation states for Entry None (Fe11O12). Assigning anion correction to only the most electronegative atom.
Failed to guess oxidation states for Entry None (Fe23O25). Assigning anion correction to only the most electronegative atom.
Duplicate entry names found for generated competing phases: Fe_Im-3m_EaH_0!


In [15]:
for key in relaxed_defect.keys():
    print(relaxed_defect[key].keys())

dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_Ci_O1.90H2.58H2.65a_0', 'Cr_Al_Ci_O1.90H2.58H2.65b_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])


In [16]:
dumpfn(relaxed_defect, f'../../data/cr_above_hull/{MODEL}_corrected_doped_defect_energies_{supercell_size}A.json')

## Leave

In [20]:
ex_cp = loadfn(f'../data/{MODEL}_doped_defect_energies_{supercell_size}A.json')

In [21]:
supercell_10A = loadfn(f'../data/{MODEL}_doped_defect_energies_10A.json')

In [24]:
for key, val in ex_cp.items():
    ex_cp[key]['minimum energy site']['energy'] = supercell_10A[key]['minimum energy site']['energy']
    ex_cp[key]['minimum energy site']['information'] = supercell_10A[key]['minimum energy site']['information']
    ex_cp[key]['supercell']['energy'] = supercell_10A[key]['supercell']['energy']

In [25]:
dumpfn(ex_cp, f'../data/{MODEL}_doped_defect_energies_10A.json')